# Importing Data Example

In order to import data into `pypillometry`, we have to load the data from the source using other packages and then wrap it into `PupilData` objects. 

Here, we will use `pandas.read_csv()`.

First, we import the needed modules.

In [4]:
import sys
sys.path.insert(0,"..") # this is not needed if you have installed pypillometry
import pypillometry as pp
import pandas as pd

In [5]:

import requests

In [6]:

requests.get()

TypeError: get() missing 1 required positional argument: 'url'